In [1]:
import numpy as np
import pandas as pd


In [2]:
df = pd.read_csv("ML-EdgeIIoT-dataset.csv")

/tmp/ipykernel_11149/3196203159.py:1: DtypeWarning: Columns (3,6,11,13,14,15,16,17,31,32,34,39,45,51,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("ML-EdgeIIoT-dataset.csv")


In [3]:
from sklearn.preprocessing import MinMaxScaler

df = df.drop(columns=['Attack_type'])

for col in df.columns:
    
    df[col] = pd.to_numeric(df[col], errors='coerce')
    
df = df.fillna(0)


In [4]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['Attack_label'])
scaler = MinMaxScaler()

numeric_columns = X.select_dtypes(include=[np.number]).columns
X[numeric_columns] = scaler.fit_transform(df[numeric_columns])

X[numeric_columns] =X[numeric_columns].astype(np.float64)

# Define target variable y
y = df['Attack_label']

# Split the data into training and testing sets (80% train, 20% test)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the training set into training and validation sets (further split to get 20% of the original data as validation)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

In [5]:
df

,frame.time,ip.src_host,ip.dst_host,arp.dst.proto_ipv4,arp.opcode,arp.hw.size,arp.src.proto_ipv4,icmp.checksum,icmp.seq_le,icmp.transmit_timestamp,...,mqtt.msgtype,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_label
0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48729.0,40690.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
157796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45657.0,40702.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
157797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57686.0,41423.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
157798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9555.0,42379.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


## Naive Bayes

In [6]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

model_nb = GaussianNB()
model_nb.fit(x_train, y_train)

y_pred_nb = model_nb.predict(x_test)
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f'Accuracy of Naive Bayes classifier: {accuracy_nb:.2f}')
report_nb = classification_report(y_test, y_pred_nb,digits=4)
print(report_nb)

Accuracy of Naive Bayes classifier: 0.86
              precision    recall  f1-score   support

           0     0.6125    0.3747    0.4650      4985
           1     0.8907    0.9555    0.9220     26575

    accuracy                         0.8638     31560
   macro avg     0.7516    0.6651    0.6935     31560
weighted avg     0.8467    0.8638    0.8498     31560



In [7]:
alphas = np.logspace(-2, 2, 5)  

best_alpha = None
best_accuracy = 0

for alpha in alphas:
    model = MultinomialNB(alpha=alpha)
    model.fit(x_train, y_train)
    val_pred = model.predict(x_val)
    accuracy = accuracy_score(y_val, val_pred)
    print(f'Alpha: {alpha}, Validation Accuracy: {accuracy:.4f}')
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_alpha = alpha

print(f'Best Alpha: {best_alpha}, Best Validation Accuracy: {best_accuracy:.4f}')

model_Mnb = MultinomialNB(alpha=best_alpha)
model_Mnb.fit(x_train, y_train)
y_pred_Mnb = model_Mnb.predict(x_test)

final_accuracy_Mnb = accuracy_score(y_test, y_pred_Mnb)
print(f'Final Accuracy of Naive Bayes classifier: {final_accuracy_Mnb:.4f}')
final_report_Mnb = classification_report(y_test, y_pred_Mnb,digits=4)
print(final_report_Mnb)

Alpha: 0.01, Validation Accuracy: 0.8896
Alpha: 0.1, Validation Accuracy: 0.8896
Alpha: 1.0, Validation Accuracy: 0.8896
Alpha: 10.0, Validation Accuracy: 0.8892
Alpha: 100.0, Validation Accuracy: 0.8770
Best Alpha: 0.01, Best Validation Accuracy: 0.8896
Final Accuracy of Naive Bayes classifier: 0.8878
              precision    recall  f1-score   support

           0     1.0000    0.2899    0.4495      4985
           1     0.8825    1.0000    0.9376     26575

    accuracy                         0.8878     31560
   macro avg     0.9412    0.6449    0.6935     31560
weighted avg     0.9010    0.8878    0.8605     31560



## Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression classifier
model_lr = LogisticRegression(max_iter=5000, tol=1e-4, verbose=1) 

# Train the model
model_lr.fit(x_train, y_train)

# Predict on the test set
y_pred_lr = model_lr.predict(x_test)

# Calculate accuracy
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f'Accuracy of Logistic Regression classifier: {accuracy_lr:.2f}')

# Print classification report
report_lr = classification_report(y_test, y_pred_lr,digits=4)
print(report_lr)

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           62     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.56272D+04    |proj g|=  3.29210D+04

At iterate   50    f=  2.01537D+04    |proj g|=  1.61799D+01

At iterate  100    f=  2.01354D+04    |proj g|=  8.26587D+00

At iterate  150    f=  2.01334D+04    |proj g|=  5.11882D-01

At iterate  200    f=  2.01334D+04    |proj g|=  3.97612D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   62    204    229      1     0     0   1.732D-01   2.013D+04
  F =   20133.388363460112     

CONVERG

In [9]:
# from sklearn.model_selection import GridSearchCV

# # Define the model
# model_lr = LogisticRegression(max_iter=1000, tol=1e-4, verbose=1)

# # Define a hyperparameter grid to search
# param_grid = {
#     'C': np.logspace(-3, 3, 7),  # example range for C
#     'penalty': ['l1', 'l2'],  # l1 lasso l2 ridge
#     'class_weight': [None, 'balanced'],  # uncomment if you have imbalanced classes
# }

# # Define the grid search
# grid_search = GridSearchCV(model_lr, param_grid, cv=5, scoring='accuracy', verbose=1)

# # Perform the grid search and fit the model
# grid_search.fit(x_train, y_train)

# # Get the results
# best_params = grid_search.best_params_
# best_estimator = grid_search.best_estimator_

# # Predict on the test set with the best params
# y_pred_lr = best_estimator.predict(x_test)

# # Calculate accuracy
# accuracy_lr = accuracy_score(y_test, y_pred_lr)
# print(f'Best Parameters: {best_params}')
# print(f'Accuracy of Logistic Regression classifier with best parameters: {accuracy_lr:.2f}')
# report_lr = classification_report(y_test, y_pred_lr)
# print(report_lr)


## SVM

In [10]:
from sklearn.svm import SVC


# Initialize the Support Vector Classifier with shrinking turned off
model_svc = SVC(shrinking=False, verbose = True)

# Train the model on your data
model_svc.fit(x_train, y_train)

# Predict on the test data
y_pred_svc = model_svc.predict(x_test)

# Calculate accuracy



[LibSVM]....................*
optimization finished, #iter = 20206
obj = -15823.015047, rho = 2.551594
nSV = 17143, nBSV = 17016
Total nSV = 17143


In [11]:
from sklearn.metrics import accuracy_score, classification_report
accuracy_svc = accuracy_score(y_test, y_pred_svc)
print(f'Accuracy of SVM classifier: {accuracy_svc:.4f}')
report_svc = classification_report(y_test, y_pred_svc,digits=4)
print(report_svc)

Accuracy of SVM classifier: 0.9102
              precision    recall  f1-score   support

           0     0.9242    0.4698    0.6230      4985
           1     0.9089    0.9928    0.9490     26575

    accuracy                         0.9102     31560
   macro avg     0.9166    0.7313    0.7860     31560
weighted avg     0.9114    0.9102    0.8975     31560



## KNN

In [12]:
from sklearn.neighbors import KNeighborsClassifier

# Initialize the k-Nearest Neighbors classifier with default parameters
model_knn = KNeighborsClassifier()

# Train the model on the training data
model_knn.fit(x_train, y_train)

# Predict on the test data
y_pred_knn = model_knn.predict(x_test)

In [13]:
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f'Accuracy of kNN classifier: {accuracy_knn:.4f}')
report_knn = classification_report(y_test, y_pred_knn,digits=4)
print(report_knn)

Accuracy of kNN classifier: 0.9680
              precision    recall  f1-score   support

           0     0.9231    0.8696    0.8956      4985
           1     0.9758    0.9864    0.9811     26575

    accuracy                         0.9680     31560
   macro avg     0.9495    0.9280    0.9383     31560
weighted avg     0.9675    0.9680    0.9676     31560



In [14]:
# Range of n_neighbors to try
neighbor_values = range(1, 20, 2)  # Example: trying odd numbers from 1 to 19

best_n_neighbors = None
best_accuracy = 0

for n in neighbor_values:
    # Initialize the kNN classifier
    model_knn = KNeighborsClassifier(n_neighbors=n)

    # Train the model on the training data
    model_knn.fit(x_train, y_train)

    # Evaluate on the validation data
    y_val_pred = model_knn.predict(x_val)
    accuracy = accuracy_score(y_val, y_val_pred)
    print(f'n_neighbors: {n}, Validation Accuracy: {accuracy:.4f}')

    # Update best parameters if accuracy is improved
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_n_neighbors = n

print(f'Best n_neighbors: {best_n_neighbors}, Best Validation Accuracy: {best_accuracy:.4f}')

# Train the final model using the best n_neighbors on the full training set
final_model_knn = KNeighborsClassifier(n_neighbors=best_n_neighbors)
final_model_knn.fit(x_train, y_train)

# Predict on the test set with the final model
y_pred_final_knn = final_model_knn.predict(x_test)


n_neighbors: 1, Validation Accuracy: 0.9796
n_neighbors: 3, Validation Accuracy: 0.9730
n_neighbors: 5, Validation Accuracy: 0.9690
n_neighbors: 7, Validation Accuracy: 0.9646
n_neighbors: 9, Validation Accuracy: 0.9615
n_neighbors: 11, Validation Accuracy: 0.9592
n_neighbors: 13, Validation Accuracy: 0.9561
n_neighbors: 15, Validation Accuracy: 0.9536
n_neighbors: 17, Validation Accuracy: 0.9517
n_neighbors: 19, Validation Accuracy: 0.9495
Best n_neighbors: 1, Best Validation Accuracy: 0.9796


In [15]:
# Final evaluation
final_accuracy_knn = accuracy_score(y_test, y_pred_final_knn)
print(f'Final Accuracy of kNN classifier: {final_accuracy_knn:.4f}')
report_svc = classification_report(y_test, y_pred_final_knn,digits=4)
print(report_svc)

Final Accuracy of kNN classifier: 0.9790
              precision    recall  f1-score   support

           0     0.9446    0.9208    0.9325      4985
           1     0.9852    0.9899    0.9875     26575

    accuracy                         0.9790     31560
   macro avg     0.9649    0.9553    0.9600     31560
weighted avg     0.9788    0.9790    0.9789     31560



## Decision Tree

In [16]:
from sklearn.tree import DecisionTreeClassifier

# Initialize the Decision Tree classifier
model_dt = DecisionTreeClassifier()

# Train the model on the training data
model_dt.fit(x_train, y_train)

# Predict on the test data
y_pred_dt = model_dt.predict(x_test)

In [17]:
# Calculate accuracy
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f'Accuracy of Decision Tree classifier: {accuracy_dt:.4f}')
report_dt = classification_report(y_test, y_pred_dt,digits=4)
print(report_dt)

Accuracy of Decision Tree classifier: 0.9992
              precision    recall  f1-score   support

           0     1.0000    0.9950    0.9975      4985
           1     0.9991    1.0000    0.9995     26575

    accuracy                         0.9992     31560
   macro avg     0.9995    0.9975    0.9985     31560
weighted avg     0.9992    0.9992    0.9992     31560



In [18]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Define the model
model_dt = DecisionTreeClassifier()

# Define a hyperparameter distribution to sample from
param_dist = {
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['auto', 'sqrt', 'log2', None]
}

# Set up the RandomizedSearchCV object
random_search = RandomizedSearchCV(model_dt, param_distributions=param_dist,
                                   n_iter=100, cv=5, scoring='accuracy', verbose=2, random_state=42)

# Perform the random search and fit the model
random_search.fit(x_train, y_train)

# Get the results
best_params = random_search.best_params_
best_accuracy = random_search.best_score_
print(f'Best Parameters: {best_params}')
print(f'Best CV Accuracy: {best_accuracy}')

# Train the final model using the best parameters on the full training set
final_model_dt = DecisionTreeClassifier(**best_params)
final_model_dt.fit(x_train, y_train)

# Predict on the test set with the final model
y_pred_final_dt = final_model_dt.predict(x_test)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END max_depth=30, max_features=auto, min_samples_leaf=8, min_samples_split=8; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=8, min_samples_split=8; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=8, min_samples_split=8; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=8, min_samples_split=8; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=8, min_samples_split=8; total time=   0.0s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=7, min_samples_split=12; total time=   0.1s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=7, min_samples_split=12; total time=   0.1s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=7, min_samples_split=12; total time=   0.1s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=7, min_samples_split=12; total time=   0.1s
[CV] END max_de

/opt/conda/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
110 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
110 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.11/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/opt/conda/lib/python3.11/site-packages/sklearn/base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "/opt/conda/lib/python3.11/site-packages/sklearn/util

Best Parameters: {'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 12}
Best CV Accuracy: 0.9993768483312209


In [19]:
# Final evaluation
final_accuracy_dt = accuracy_score(y_test, y_pred_final_dt)
print(f'Final Accuracy of Decision Tree classifier: {final_accuracy_dt:.4f}')
report_final_dt = classification_report(y_test, y_pred_final_dt,digits=4)
print(report_final_dt)

Final Accuracy of Decision Tree classifier: 0.9992
              precision    recall  f1-score   support

           0     0.9998    0.9950    0.9974      4985
           1     0.9991    1.0000    0.9995     26575

    accuracy                         0.9992     31560
   macro avg     0.9994    0.9975    0.9984     31560
weighted avg     0.9992    0.9992    0.9992     31560



## Random Forest

In [20]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest classifier
model_rf = RandomForestClassifier()

# Train the model on the training data
model_rf.fit(x_train, y_train)

# Predict on the test data
y_pred_rf = model_rf.predict(x_test)

# Calculate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f'Accuracy of Random Forest classifier: {accuracy_rf:.4f}')
report_rf = classification_report(y_test, y_pred_rf,digits=4)
print(report_rf)

Accuracy of Random Forest classifier: 0.9991
              precision    recall  f1-score   support

           0     1.0000    0.9946    0.9973      4985
           1     0.9990    1.0000    0.9995     26575

    accuracy                         0.9991     31560
   macro avg     0.9995    0.9973    0.9984     31560
weighted avg     0.9991    0.9991    0.9991     31560



## XGBoost

In [23]:
import xgboost as xgb

# Initialize the XGBoost classifier
model_xgb = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Train the model on the training data
model_xgb.fit(x_train, y_train)

# Predict on the test data
y_pred_xgb = model_xgb.predict(x_test)

# Calculate accuracy
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f'Accuracy of XGBoost classifier: {accuracy_xgb:.4f}')
report_xgb = classification_report(y_test, y_pred_xgb,digits=4)
print(report_xgb)

Accuracy of XGBoost classifier: 0.9991
              precision    recall  f1-score   support

           0     1.0000    0.9942    0.9971      4985
           1     0.9989    1.0000    0.9995     26575

    accuracy                         0.9991     31560
   macro avg     0.9995    0.9971    0.9983     31560
weighted avg     0.9991    0.9991    0.9991     31560



In [24]:
best_accuracy = 0
best_params = {}

for n_estimators in [100, 200, 300]:
    for max_depth in [3, 5, 7]:
        for learning_rate in [0.01, 0.1, 0.2]:
            model = xgb.XGBClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                learning_rate=learning_rate,
                use_label_encoder=False,
                eval_metric='mlogloss'
            )
            model.fit(x_train, y_train)
            y_pred_val = model.predict(x_val)
            accuracy = accuracy_score(y_val, y_pred_val)
            print(f"n_estimators: {n_estimators}, max_depth: {max_depth}, learning_rate: {learning_rate}, Accuracy: {accuracy:.4f}")

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {'n_estimators': n_estimators, 'max_depth': max_depth, 'learning_rate': learning_rate}

print(f"Best Parameters: {best_params}, Best Accuracy: {best_accuracy:.4f}")

n_estimators: 100, max_depth: 3, learning_rate: 0.01, Accuracy: 0.9715
n_estimators: 100, max_depth: 3, learning_rate: 0.1, Accuracy: 0.9988
n_estimators: 100, max_depth: 3, learning_rate: 0.2, Accuracy: 0.9988
n_estimators: 100, max_depth: 5, learning_rate: 0.01, Accuracy: 0.9907
n_estimators: 100, max_depth: 5, learning_rate: 0.1, Accuracy: 0.9990
n_estimators: 100, max_depth: 5, learning_rate: 0.2, Accuracy: 0.9990
n_estimators: 100, max_depth: 7, learning_rate: 0.01, Accuracy: 0.9984
n_estimators: 100, max_depth: 7, learning_rate: 0.1, Accuracy: 0.9990
n_estimators: 100, max_depth: 7, learning_rate: 0.2, Accuracy: 0.9990
n_estimators: 200, max_depth: 3, learning_rate: 0.01, Accuracy: 0.9895
n_estimators: 200, max_depth: 3, learning_rate: 0.1, Accuracy: 0.9988
n_estimators: 200, max_depth: 3, learning_rate: 0.2, Accuracy: 0.9990
n_estimators: 200, max_depth: 5, learning_rate: 0.01, Accuracy: 0.9988
n_estimators: 200, max_depth: 5, learning_rate: 0.1, Accuracy: 0.9990
n_estimators: 2

In [25]:
# Retrain the model with the best parameters on the full training set
final_model_xgb = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='mlogloss')
final_model_xgb.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.2, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [26]:

# Predict on the test set
y_pred_test = final_model_xgb.predict(x_test)

# Calculate and print the accuracy
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Test Set Accuracy: {test_accuracy:.4f}")

# Generate and print the classification report
report = classification_report(y_test, y_pred_test,digits=4)
print("Classification Report:\n", report)

Test Set Accuracy: 0.9991
Classification Report:
               precision    recall  f1-score   support

           0     1.0000    0.9942    0.9971      4985
           1     0.9989    1.0000    0.9995     26575

    accuracy                         0.9991     31560
   macro avg     0.9995    0.9971    0.9983     31560
weighted avg     0.9991    0.9991    0.9991     31560



## Score Report

### Naive Bayes

In [27]:
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score
y_true_Mnb = y_test
y_pred_Mnb = model_Mnb.predict(x_test)


tn_Mnb, fp_Mnb, fn_Mnb, tp_Mnb = confusion_matrix(y_true_Mnb, y_pred_Mnb).ravel()

accuracy_Mnb = accuracy_score(y_true_Mnb, y_pred_Mnb)
precision_Mnb = precision_score(y_true_Mnb, y_pred_Mnb)
recall_Mnb = recall_score(y_true_Mnb, y_pred_Mnb)  
f1_Mnb = f1_score(y_true_Mnb, y_pred_Mnb)
auc_Mnb = roc_auc_score(y_true_Mnb, y_pred_Mnb)

fpr_Mnb = fp_Mnb / (fp_Mnb + tn_Mnb)
fnr_Mnb = fn_Mnb / (fn_Mnb + tp_Mnb)
tnr_Mnb = tn_Mnb / (tn_Mnb + fp_Mnb)

print(f"Accuracy of MultinomialNB: {accuracy_Mnb:.4f}")
print(f"Precision of MultinomialNB: {precision_Mnb:.4f}")
print(f"Recall (TPR) of MultinomialNB: {recall_Mnb:.4f}")
print(f"F1 Score of MultinomialNB: {f1_Mnb:.4f}")
print(f"AUC of MultinomialNB: {auc_Mnb:.4f}")
print(f"FPR of MultinomialNB: {fpr_Mnb:.4f}")
print(f"FNR of MultinomialNB: {fnr_Mnb:.4f}")
print(f"TNR of MultinomialNB: {tnr_Mnb:.4f}")

Accuracy of MultinomialNB: 0.8878
Precision of MultinomialNB: 0.8825
Recall (TPR) of MultinomialNB: 1.0000
F1 Score of MultinomialNB: 0.9376
AUC of MultinomialNB: 0.6449
FPR of MultinomialNB: 0.7101
FNR of MultinomialNB: 0.0000
TNR of MultinomialNB: 0.2899


### Logistic Regression

In [28]:
y_true_lr = y_test
y_pred_lr = model_lr.predict(x_test)

# Confusion matrix components
tn_lr, fp_lr, fn_lr, tp_lr = confusion_matrix(y_true_lr, y_pred_lr).ravel()

# Calculating metrics
accuracy_lr = accuracy_score(y_true_lr, y_pred_lr)
precision_lr = precision_score(y_true_lr, y_pred_lr)
recall_lr = recall_score(y_true_lr, y_pred_lr)  # Also known as TPR
f1_lr = f1_score(y_true_lr, y_pred_lr)
auc_lr = roc_auc_score(y_true_lr, y_pred_lr)

# FPR, FNR, TNR
fpr_lr = fp_lr / (fp_lr + tn_lr)
fnr_lr = fn_lr / (fn_lr + tp_lr)
tnr_lr = tn_lr / (tn_lr + fp_lr)

# Printing the metrics
print(f"Accuracy of Logistic Regression: {accuracy_lr:.4f}")
print(f"Precision of Logistic Regression: {precision_lr:.4f}")
print(f"Recall (TPR) of Logistic Regression: {recall_lr:.4f}")
print(f"F1 Score of Logistic Regression: {f1_lr:.4f}")
print(f"AUC of Logistic Regression: {auc_lr:.4f}")
print(f"FPR of Logistic Regression: {fpr_lr:.4f}")
print(f"FNR of Logistic Regression: {fnr_lr:.4f}")
print(f"TNR of Logistic Regression: {tnr_lr:.4f}")

Accuracy of Logistic Regression: 0.9122
Precision of Logistic Regression: 0.9141
Recall (TPR) of Logistic Regression: 0.9887
F1 Score of Logistic Regression: 0.9499
AUC of Logistic Regression: 0.7466
FPR of Logistic Regression: 0.4955
FNR of Logistic Regression: 0.0113
TNR of Logistic Regression: 0.5045


### SVM

In [29]:
y_true_svc = y_test
y_pred_svc = model_svc.predict(x_test)

# Confusion matrix components
tn_svc, fp_svc, fn_svc, tp_svc = confusion_matrix(y_true_svc, y_pred_svc).ravel()

# Calculating metrics
accuracy_svc = accuracy_score(y_true_svc, y_pred_svc)
precision_svc = precision_score(y_true_svc, y_pred_svc)
recall_svc = recall_score(y_true_svc, y_pred_svc)  # Also known as TPR
f1_svc = f1_score(y_true_svc, y_pred_svc)
auc_svc = roc_auc_score(y_true_svc, y_pred_svc)

# FPR, FNR, TNR
fpr_svc = fp_svc / (fp_svc + tn_svc)
fnr_svc = fn_svc / (fn_svc + tp_svc)
tnr_svc = tn_svc / (tn_svc + fp_svc)

# Printing the metrics
print(f"Accuracy of SVM: {accuracy_svc:.4f}")
print(f"Precision of SVM: {precision_svc:.4f}")
print(f"Recall (TPR) of SVM: {recall_svc:.4f}")
print(f"F1 Score of SVM: {f1_svc:.4f}")
print(f"AUC of SVM: {auc_svc:.4f}")
print(f"FPR of SVM: {fpr_svc:.4f}")
print(f"FNR of SVM: {fnr_svc:.4f}")
print(f"TNR of SVM: {tnr_svc:.4f}")

Accuracy of SVM: 0.9102
Precision of SVM: 0.9089
Recall (TPR) of SVM: 0.9928
F1 Score of SVM: 0.9490
AUC of SVM: 0.7313
FPR of SVM: 0.5302
FNR of SVM: 0.0072
TNR of SVM: 0.4698


### KNN

In [30]:
y_true_final_knn = y_test
y_pred_final_knn = final_model_knn.predict(x_test)

# Confusion matrix components for the final model
tn_final_knn, fp_final_knn, fn_final_knn, tp_final_knn = confusion_matrix(y_true_final_knn, y_pred_final_knn).ravel()

# Calculating metrics for the final model
accuracy_final_knn = accuracy_score(y_true_final_knn, y_pred_final_knn)
precision_final_knn = precision_score(y_true_final_knn, y_pred_final_knn)
recall_final_knn = recall_score(y_true_final_knn, y_pred_final_knn)  # TPR
f1_final_knn = f1_score(y_true_final_knn, y_pred_final_knn)
auc_final_knn = roc_auc_score(y_true_final_knn, y_pred_final_knn)

# FPR, FNR, TNR for the final model
fpr_final_knn = fp_final_knn / (fp_final_knn + tn_final_knn)
fnr_final_knn = fn_final_knn / (fn_final_knn + tp_final_knn)
tnr_final_knn = tn_final_knn / (tn_final_knn + fp_final_knn)

# Printing the metrics for the final model
print(f"Final Accuracy of KNN: {accuracy_final_knn:.4f}")
print(f"Final Precision of KNN: {precision_final_knn:.4f}")
print(f"Final Recall (TPR) of KNN: {recall_final_knn:.4f}")
print(f"Final F1 Score of KNN: {f1_final_knn:.4f}")
print(f"Final AUC of KNN: {auc_final_knn:.4f}")
print(f"Final FPR of KNN: {fpr_final_knn:.4f}")
print(f"Final FNR of KNN: {fnr_final_knn:.4f}")
print(f"Final TNR of KNN: {tnr_final_knn:.4f}")

Final Accuracy of KNN: 0.9790
Final Precision of KNN: 0.9852
Final Recall (TPR) of KNN: 0.9899
Final F1 Score of KNN: 0.9875
Final AUC of KNN: 0.9553
Final FPR of KNN: 0.0792
Final FNR of KNN: 0.0101
Final TNR of KNN: 0.9208


### Decision Tree

In [31]:
y_true_final_dt = y_test
y_pred_final_dt = final_model_dt.predict(x_test)

# Confusion matrix components for the final model
tn_final_dt, fp_final_dt, fn_final_dt, tp_final_dt = confusion_matrix(y_true_final_dt, y_pred_final_dt).ravel()

# Calculating metrics for the final model
accuracy_final_dt = accuracy_score(y_true_final_dt, y_pred_final_dt)
precision_final_dt = precision_score(y_true_final_dt, y_pred_final_dt)
recall_final_dt = recall_score(y_true_final_dt, y_pred_final_dt)  # TPR
f1_final_dt = f1_score(y_true_final_dt, y_pred_final_dt)
auc_final_dt = roc_auc_score(y_true_final_dt, y_pred_final_dt)

# FPR, FNR, TNR for the final model
fpr_final_dt = fp_final_dt / (fp_final_dt + tn_final_dt)
fnr_final_dt = fn_final_dt / (fn_final_dt + tp_final_dt)
tnr_final_dt = tn_final_dt / (tn_final_dt + fp_final_dt)

# Printing the metrics for the final model
print(f"Final Accuracy of DT: {accuracy_final_dt:.4f}")
print(f"Final Precision of DT: {precision_final_dt:.4f}")
print(f"Final Recall (TPR) of DT: {recall_final_dt:.4f}")
print(f"Final F1 Score of DT: {f1_final_dt:.4f}")
print(f"Final AUC of DT: {auc_final_dt:.4f}")
print(f"Final FPR of DT: {fpr_final_dt:.4f}")
print(f"Final FNR of DT: {fnr_final_dt:.4f}")
print(f"Final TNR of DT: {tnr_final_dt:.4f}")

Final Accuracy of DT: 0.9992
Final Precision of DT: 0.9991
Final Recall (TPR) of DT: 1.0000
Final F1 Score of DT: 0.9995
Final AUC of DT: 0.9975
Final FPR of DT: 0.0050
Final FNR of DT: 0.0000
Final TNR of DT: 0.9950


### Random Forest

In [32]:
y_true_final_rf = y_test
y_pred_final_rf = model_rf.predict(x_test)

# Confusion matrix components for the final model
tn_final_rf, fp_final_rf, fn_final_rf, tp_final_rf = confusion_matrix(y_true_final_rf, y_pred_final_rf).ravel()

# Calculating metrics for the final model
accuracy_final_rf = accuracy_score(y_true_final_rf, y_pred_final_rf)
precision_final_rf = precision_score(y_true_final_rf, y_pred_final_rf)
recall_final_rf = recall_score(y_true_final_rf, y_pred_final_rf)  # TPR
f1_final_rf = f1_score(y_true_final_rf, y_pred_final_rf)
auc_final_rf = roc_auc_score(y_true_final_rf, y_pred_final_rf)

# FPR, FNR, TNR for the final model
fpr_final_rf = fp_final_rf / (fp_final_rf + tn_final_rf)
fnr_final_rf = fn_final_rf / (fn_final_rf + tp_final_rf)
tnr_final_rf = tn_final_rf / (tn_final_rf + fp_final_rf)

# Printing the metrics for the final model
print(f"Final Accuracy of RF: {accuracy_final_rf:.4f}")
print(f"Final Precision of RF: {precision_final_rf:.4f}")
print(f"Final Recall (TPR) of RF: {recall_final_rf:.4f}")
print(f"Final F1 Score of RF: {f1_final_rf:.4f}")
print(f"Final AUC of RF: {auc_final_rf:.4f}")
print(f"Final FPR of RF: {fpr_final_rf:.4f}")
print(f"Final FNR of RF: {fnr_final_rf:.4f}")
print(f"Final TNR of RF: {tnr_final_rf:.4f}")


Final Accuracy of RF: 0.9991
Final Precision of RF: 0.9990
Final Recall (TPR) of RF: 1.0000
Final F1 Score of RF: 0.9995
Final AUC of RF: 0.9973
Final FPR of RF: 0.0054
Final FNR of RF: 0.0000
Final TNR of RF: 0.9946


### XGB

In [33]:
y_true_final_xgb = y_test
y_pred_final_xgb = final_model_xgb.predict(x_test)

# Confusion matrix components for the final model
tn_final_xgb, fp_final_xgb, fn_final_xgb, tp_final_xgb = confusion_matrix(y_true_final_xgb, y_pred_final_xgb).ravel()

# Calculating metrics for the final model
accuracy_final_xgb = accuracy_score(y_true_final_xgb, y_pred_final_xgb)
precision_final_xgb = precision_score(y_true_final_xgb, y_pred_final_xgb)
recall_final_xgb = recall_score(y_true_final_xgb, y_pred_final_xgb)  # TPR
f1_final_xgb = f1_score(y_true_final_xgb, y_pred_final_xgb)
auc_final_xgb = roc_auc_score(y_true_final_xgb, y_pred_final_xgb)

# FPR, FNR, TNR for the final model
fpr_final_xgb = fp_final_xgb / (fp_final_xgb + tn_final_xgb)
fnr_final_xgb = fn_final_xgb / (fn_final_xgb + tp_final_xgb)
tnr_final_xgb = tn_final_xgb / (tn_final_xgb + fp_final_xgb)

# Printing the metrics for the final model
print(f"Final Accuracy of XGB: {accuracy_final_xgb:.4f}")
print(f"Final Precision of XGB: {precision_final_xgb:.4f}")
print(f"Final Recall (TPR) of XGB: {recall_final_xgb:.4f}")
print(f"Final F1 Score of XGB: {f1_final_xgb:.4f}")
print(f"Final AUC of XGB: {auc_final_xgb:.4f}")
print(f"Final FPR of XGB: {fpr_final_xgb:.4f}")
print(f"Final FNR of XGB: {fnr_final_xgb:.4f}")
print(f"Final TNR of XGB: {tnr_final_xgb:.4f}")

Final Accuracy of XGB: 0.9991
Final Precision of XGB: 0.9989
Final Recall (TPR) of XGB: 1.0000
Final F1 Score of XGB: 0.9995
Final AUC of XGB: 0.9971
Final FPR of XGB: 0.0058
Final FNR of XGB: 0.0000
Final TNR of XGB: 0.9942


## Transfer

In [50]:
epsilon = 0.01  # Specify the epsilon value corresponding to the file you want to load
filename = f'x_test_adv_eps_{epsilon}.npy'
X_test_adv = np.load(filename)
y_test_indicies = np.load('y_test_indices.npy')
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming 'model' is your trained traditional ML model
y_pred_lr=model_lr.predict(X_test_adv)

# Calculate precision, recall, and F1 score
precision_micro, recall_micro, fscore_micro, _ = precision_recall_fscore_support(y_test_indices, preds_FGSM, average='micro')
precision_macro, recall_macro, fscore_macro, _ = precision_recall_fscore_support(y_test_indices, preds_FGSM, average='macro')

    # Calculate accuracy for reference
acc = np.sum(preds_FGSM == y_test_indices) / y_test_indices.shape[0]

    # Calculate confusion matrix and then TPR, TNR, FPR, FNR for each class
conf_matrix = confusion_matrix(y_test_indices, preds_FGSM)
FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix)  
FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
TP = np.diag(conf_matrix)
TN = conf_matrix.sum() - (FP + FN + TP)

TPR = TP / (TP + FN)
TNR = TN / (TN + FP)
FPR = FP / (FP + TN)
FNR = FN / (TP + FN)

    # Averaging TPR, TNR, FPR, FNR
TPR_avg = np.mean(TPR)
TNR_avg = np.mean(TNR)
FPR_avg = np.mean(FPR)
FNR_avg = np.mean(FNR)
TP_micro = np.sum(TP)
FP_micro = np.sum(FP)
FN_micro = np.sum(FN)
TN_micro = np.sum(TN)

    # Micro averages for applicable metrics
TPR_micro = TP_micro / (TP_micro + FN_micro)
FPR_micro = FP_micro / (FP_micro + TN_micro)
    
    # Print the results for each epsilon
print(f"Epsilon: {epsilon}")
print(f"Accuracy: {acc}")
print(f"Micro Precision: {precision_micro:.2f}")
print(f"Macro Precision: {precision_macro:.2f}")
print(f"Micro Recall: {recall_micro:.2f}")
print(f"Macro Recall: {recall_macro:.2f}")
print(f"Micro F1 Score: {fscore_micro:.2f}")
print(f"Macro F1 Score: {fscore_macro:.2f}")
print(f"Average TNR: {TNR_avg:.2f}, Average FPR: {FPR_avg:.2f}, Average FNR: {FNR_avg:.2f}\n")
print(f"Macro TNR: {TNR_avg:.2f}, Macro FNR: {FNR_avg:.2f},Macro FPR: {FPR_avg:.2f}")
print()


/opt/conda/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [52]:
from joblib import dump
from joblib import load
dump(model_lr, 'model_lr.joblib')
dump(model_nb, 'model_nb.joblib')
dump(model_svc, 'model_svc.joblib')
dump(final_model_knn, 'model_knn.joblib')
dump(final_model_dt, 'model_dt.joblib')
dump(model_rf, 'model_rf.joblib')
dump(final_model_xgb, 'model_xgb.joblib')

['model_xgb.joblib']